In [2]:
import pandas as pd
import utils 
import plotly.graph_objects as go 

In [3]:
pair = "EUR_USD"
granularity = 'H1'
ma_list = [16,64]

In [4]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity) )

In [5]:
non_cols =['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

set mod_cols = to a list and get us each value in the  dataframe column if that 'x' is not in the list, then create a new list 
this happens often that we need to convert strings to numbers or any other data type 

In [6]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [7]:
df_ma.head(5)

,time,mid_o,mid_h,mid_l,mid_c
0,2023-12-06T08:00:00.000000000Z,1.07898,1.07904,1.07750,1.07868
1,2023-12-06T09:00:00.000000000Z,1.07869,1.07979,1.07860,1.07911
2,2023-12-06T10:00:00.000000000Z,1.07912,1.07960,1.07874,1.07911
3,2023-12-06T11:00:00.000000000Z,1.07909,1.07909,1.07780,1.07800
4,2023-12-06T12:00:00.000000000Z,1.07800,1.07808,1.07724,1.07774


In [8]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

AttributeError: 'DataFrame' object has no attribute 'MA_16'

In [ ]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [ ]:
def is_trade(row):
    if row.DIFF >= 0  and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0  and row.DIFF_PREV > 0:
        return True
    return False

In [ ]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

axis =1 means we are going row by row 
we are calling IS_TRADE for each row

In [ ]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [ ]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
185,2023-12-18T01:00:00.000000000Z,1.09005,1.09008,1.08946,1.08976,1.091350,1.091535,-0.000185,0.000379,True
222,2023-12-19T14:00:00.000000000Z,1.09620,1.09844,1.09592,1.09792,1.093654,1.093307,0.000348,-0.000025,True
263,2023-12-21T07:00:00.000000000Z,1.09451,1.09503,1.09424,1.09472,1.094874,1.095046,-0.000173,0.000046,True
269,2023-12-21T13:00:00.000000000Z,1.09804,1.10026,1.09726,1.09911,1.095458,1.095451,0.000007,-0.000186,True
380,2023-12-29T04:00:00.000000000Z,1.10654,1.10688,1.10638,1.10668,1.107342,1.107549,-0.000207,0.000163,True


In [10]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [11]:
df_ma.dropna(inplace=True)

In [12]:
df_ma.reset_index(drop=True, inplace=True)

In [13]:
df_plot = df_ma.iloc[100:200].copy()

In [14]:
df_plot.shape

(100, 7)

In [15]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
    ))
fig.update_layout(width=1350,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()